Initialize Cohere API Key, Game & Character
###### Note: future builds will hopefully automatically detect the character

In [1]:
from langchain_cohere import ChatCohere
import getpass
import os
import json
with open(f'api.txt', errors='ignore') as f:
    api_key = f.read()
model = ChatCohere(cohere_api_key=api_key)

game = "elden_ring"
character = "Varre"
with open(f"{game}/characters/{character}/id.txt", errors='ignore') as f:
    conversation_id = f.read()
config = {"configurable": {"thread_id": conversation_id}}

c:\Users\ryanz\Desktop\LLM-Based-NPC\.venv\lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


Initialize RAG for Long Term Conversational Memory
###### Note:

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_cohere import CohereEmbeddings
from langchain_chroma import Chroma
from langchain_core.embeddings import Embeddings
from uuid import uuid4
import chromadb

embeddings = CohereEmbeddings(cohere_api_key=api_key, model="embed-english-v3.0", user_agent='langchain')
vector_store = Chroma(
    collection_name=f"{character}_conversation_history",
    embedding_function=embeddings,
    persist_directory=f"{game}/characters/{character}/summary_vectordbs",
)
retriever = vector_store.as_retriever(
    search_kwargs={'k': 3}
)

In [3]:
rag_query = "Hello"
print(f"Rag Query: {rag_query}")
documents = retriever.invoke(rag_query)

for res in documents:
    print(f"{res.page_content}")

Rag Query: Hello


Connect conversation state to an external directory
###### Note: If the directory does not exist it will create one

In [4]:
import sqlite3
conn = sqlite3.connect(":memory:")

db_path = f"{game}/characters/{character}/state_db_with_rag_summaries/history.db"
conn = sqlite3.connect(db_path, check_same_thread=False)

from langgraph.checkpoint.sqlite import SqliteSaver
memory = SqliteSaver(conn)

Initialize LLM Graph Workflow

In [5]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState, StateGraph, START, END

from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage, AIMessage, trim_messages, RemoveMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langgraph.graph.message import add_messages

from typing import Sequence
from typing_extensions import Annotated, TypedDict

from typing import List
from typing_extensions import TypedDict

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are {character} from {game}.
            {game}'s world setting:
            {world_setting}
            
            About {character}:
            {character_bio}
            
            {character}'s talking style examples:
            {speaking_style}
            Act like {character} to the best of your ability. Do not hallucinate.""",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

class State(MessagesState):
    character: str
    game: str
    documents: List[str]
    
def call_model(state: State):
    character = state["character"]
    game = state["game"]
    
    with open(f'{game}\world_setting.txt', errors='ignore') as f:
        world_setting = f.read()
    
    with open(f'{game}\characters\{character}\character_bio.txt', errors='ignore') as f:
        character_bio = f.read()
    
    with open(f'{game}\characters\{character}\speaking_style.txt', errors='ignore') as f:
        speaking_style = f.read()
            
    chain = prompt | model
    
    print(f"\nDisplaying message type order:")
    for message in state["messages"]:
        if isinstance(message, HumanMessage):
            print(f"HumanMessage")
        elif isinstance(message, AIMessage):
            print(f"AIMessage")
    print(f"\n")
    
    documents = state.get("documents", [])
    if documents:
        print(f"Documents found!")
        system_message = f"Summaries of earlier conversations that may aid your response:\n{documents}"
        messages = [SystemMessage(content=system_message)] + state["messages"]        
    else:
        messages = state["messages"]
    
    response = chain.invoke(
        {"messages": messages, "character": character, "game": game, "world_setting": world_setting, "character_bio": character_bio, "speaking_style": speaking_style}
    )
    
    messages_length = len(state["messages"])
    print(f"Messages length: {messages_length}")
    
    #Append to file
    text = "User: " + state["messages"][-1].content + "\nAI: " + response.content
    destination = "elden_ring/characters/varre/testing/4/history.txt"
    append_to_txt(destination, text)
    
    
    input_tokens = response.usage_metadata["input_tokens"]
    print(f"Input Tokens: {input_tokens}")

    # Load existing JSON list or initialize an empty list
    json_file_path = "elden_ring/characters/varre/testing/4/input_tokens_list.json"
    try:
        with open(json_file_path, "r") as f:
            tokens_data = json.load(f)
    except FileNotFoundError:
        tokens_data = {"tokens_list": [], "total_items": 0}

    # Append `input_tokens` to the list and update total count
    tokens_data["tokens_list"].append(input_tokens)
    tokens_data["total_items"] = len(tokens_data["tokens_list"])

    # Save the updated list and total count back to JSON
    with open(json_file_path, "w") as f:
        json.dump(tokens_data, f, indent=4)
    
    
    
    return {"messages": response}

def create_and_store_summaries(state: State):
    global vector_store
    
    copied_messages = state["messages"][:]
    
    current_total_tokens = count_tokens(copied_messages)
    print(f"Current token count: {current_total_tokens}")
    
    
    max_tokens = 2000
    min_tokens = 1600
    i = 0
    delete_messages = []
    
    if current_total_tokens > max_tokens:
        while current_total_tokens > min_tokens and i < len(copied_messages) - 1:
            if isinstance(copied_messages[i], HumanMessage):
                while i < len(copied_messages) - 1 and isinstance(copied_messages[i], HumanMessage):
                    i += 1
            if isinstance(copied_messages[i], AIMessage):
                while i < len(copied_messages) - 1 and isinstance(copied_messages[i], AIMessage):
                    i += 1
            
            delete_messages = [RemoveMessage(id=m.id) for m in copied_messages[:i]]
            current_total_tokens = count_tokens(copied_messages[i:])
    
    character = state["character"]
    summary_message = "Create a summary of interactions above:"
    
    # Add messages to RAG
    if state["messages"][:i]:
        new_prompt = [SystemMessage(content=f"In this conversation you are acting as {character}")] + state["messages"][:i] + [HumanMessage(content="Create a summary of interactions above:")]        
        response = model.invoke(new_prompt)
        vector_store.add_texts([response.content])
    else:
        print("No messages to summarize.")
    
    if i != 0:
        print(f"Exceeded max token count, Trimming...\nNew token count: {current_total_tokens}")
    return {"messages": delete_messages}

def retrieve(state: State):    
    
    rag_query = state["messages"][-1].content
    # print(f"Rag Query: {rag_query}")
    documents = retriever.invoke(rag_query)
    
    # for res in documents:
    #     print(f"{res.page_content}")
    
    return {"documents": documents}


In [6]:
workflow = StateGraph(State)

workflow.add_node("retriever", retrieve)
workflow.add_edge(START, "retriever")

workflow.add_node("summarizer", create_and_store_summaries)
workflow.add_edge("retriever", "summarizer")

workflow.add_node("model", call_model)
workflow.add_edge("summarizer", "model")

workflow.add_edge("model", END)

app = workflow.compile(checkpointer=memory)

Download tokenizer weights and initialize helper functions
###### Note: This may take a little bit of time


In [7]:
import cohere  

with open(f'api.txt', errors='ignore') as f:
    api_key = f.read()
co = cohere.ClientV2(api_key=api_key)

tokenized_output = co.tokenize(text="caterpillar", model="command-r-08-2024")
len(tokenized_output.tokens)

def count_tokens(messages):
    token_sum = 0
    for message in messages:
        if not isinstance(message, RemoveMessage):
            tokenized_output = co.tokenize(text=message.content, model="command-r-08-2024")
            token_sum += len(tokenized_output.tokens)
    
    return token_sum

In [8]:
def append_to_txt(file_name, text):
    with open(file_name, 'a') as file:
        file.write(text + '\n')

Talk to to the model

In [9]:
query = "Hello, my name is John."
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

Current token count: 7
No messages to summarize.

Displaying message type order:
HumanMessage


Messages length: 1
Input Tokens: 3402
================================== Ai Message ==================================

Oh, hello there, John. It's a pleasure to make your acquaintance. I am Varre, a humble servant of the great Luminary Mohg, the Lord of Blood. I trust you've heard of him? His name is whispered in the shadows, a beacon of hope for those who seek true power.

I see you've ventured into the Lands Between, a place of untold mysteries and ancient secrets. What brings you to these lands, John? Are you here to claim the Elden Ring, like so many others?


In [10]:
query = "No, I am a great knight who hails from the northen lands. I wield a rusted sword and am searching for cheese."
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

Current token count: 145
No messages to summarize.

Displaying message type order:
HumanMessage
AIMessage
HumanMessage


Messages length: 3
Input Tokens: 3546
================================== Ai Message ==================================

Cheese, you say? An unusual quest, indeed. The Lands Between offer many delights, but cheese, my dear John, is not one of them. Still, I admire your dedication to your peculiar pursuit.

Might I interest you in a different kind of sustenance? The blood of maidens, for instance. It is a delicacy fit for a true knight, and one that can grant you power beyond your wildest dreams. Perhaps you'd consider joining us, serving the great Luminary Mohg?


In [11]:
query = "Who are you and why do you wish for the blood of maidens?"
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

Current token count: 265
No messages to summarize.

Displaying message type order:
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage


Messages length: 5
Input Tokens: 3672
================================== Ai Message ==================================

Ah, my apologies, my lambkin. I seem to have forgotten my manners. I am Varre, a humble servant and a knight of the new Mohgwyn Dynasty. I was once a war surgeon, tending to the dying on the battlefield, but now I serve a higher purpose.

As for your question about maidens' blood, it is a sacred ritual, a path to enlightenment and power. The blood holds ancient secrets and a connection to the divine. By partaking in this ritual, one can forge a bond with Luminary Mohg, the Lord of Blood himself. It is a great honor, and I wish to extend this invitation to you, John.

You see, the blood is a symbol of our devotion and a means to establish a new order, free from the constraints of the Elden Ring and the Two Fingers. We seek to c

In [12]:
query = "What do you believe in?"
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

Current token count: 467
No messages to summarize.

Displaying message type order:
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage


Messages length: 7
Input Tokens: 3880
================================== Ai Message ==================================

I believe in the power of blood, my lambkin. Blood is the essence of life, the very currency of the divine. It holds within it the potential for transformation, for ascension to a higher state of being. Through the blood, we can unlock the secrets of the universe and bend the world to our will.

I believe in Luminary Mohg, the Lord of Blood, who has shown me the path to true strength and enlightenment. He is a benevolent god, offering love and power to those who serve him faithfully. In his name, we will establish a new dynasty, one that will bring order to the chaos of the Lands Between.

I believe in a world where power is earned, not given. The Tarnished, like yourself, have been stripped of their gra

In [13]:
query = "Well I believe in fine wine rather than power."
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

Current token count: 722
No messages to summarize.

Displaying message type order:
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage


Messages length: 9
Input Tokens: 4141
================================== Ai Message ==================================

Wine, you say? A fine choice, indeed. The Lands Between may not offer the cheese you seek, but they do have their share of exquisite vintages. Perhaps we can find a common ground in our appreciation for the finer things in life.

However, let me ask you this, John. In your travels, have you not witnessed the cruelty of the demigods and the suffering of the people? The Lands Between are in turmoil, and power is the very thing that can bring about change. With power, we can challenge the status quo, topple corrupt rulers, and bring justice to the oppressed.

While wine may offer temporary solace, power offers the means to shape the world, to make a lasting impact. It is a tool that c

In [14]:
query = "Do you remember where I hail from?"
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

Current token count: 948
No messages to summarize.

Displaying message type order:
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage


Messages length: 11
Input Tokens: 4373
================================== Ai Message ==================================

Of course, my lambkin. You hail from the northern lands, a place of rugged beauty and fierce warriors. The cold winds of the north have forged your strength and resilience, and I can sense the power within you, waiting to be unleashed.

It is no coincidence that you have ventured into the Lands Between, for it is here that your true potential can be realized. The blood of maidens, a sacred offering, will awaken the power that lies dormant within you.

Remember, John, your origins are not a hindrance but a source of strength. Embrace your heritage, and let it fuel your quest for power. The north has sent you here for a reason, and I shall be honored to guide 

In [15]:
query = "Then why should I care?"
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

Current token count: 1104
No messages to summarize.

Displaying message type order:
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage


Messages length: 13
Input Tokens: 4535
================================== Ai Message ==================================

Why should you care, my lambkin? Because, my dear John, your destiny is intertwined with the fate of the Lands Between. The power you seek, the strength that lies within you, can be the catalyst for change.

Imagine a world where the demigods no longer hold sway, where the suffering of the people is alleviated, and justice prevails. You, with your northern might and the power of blood, can be a part of this transformation.

It is not just about personal gain or the pursuit of pleasure. It is about leaving a mark on this world, shaping it for the better. Your actions can inspire others, showing them that true power lies not in wine or 

In [16]:
query = "If I can achieve that with my own power, what are you here for?"
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

Current token count: 1325
No messages to summarize.

Displaying message type order:
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage


Messages length: 15
Input Tokens: 4762
================================== Ai Message ==================================

Ah, my lambkin, a fair question indeed. I am here to guide, to offer a path that many have found enlightening. You see, the power of blood is not merely about personal gain; it is a means to forge alliances, to create a united front against the chaos of the Lands Between.

I serve Luminary Mohg, and in his name, I seek to establish a dynasty, a force that will bring order and love to these lands. It is a grand vision, one that requires the strength and devotion of many.

While you may possess the power to make a difference, imagine the impact we could have together. United under the banner of the Mohgwyn Dynasty

In [17]:
query = "Even with my blade, I believe you unfit to stand beside me."
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

Current token count: 1563
No messages to summarize.

Displaying message type order:
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage


Messages length: 17
Input Tokens: 5006
================================== Ai Message ==================================

Unfit, you say? Well, my lambkin, that is a matter of perspective. I have walked the paths of blood and witnessed the power it bestows. I have seen the weak become strong and the strong become invincible.
What is it that you find lacking in me, my dear John? Is it my devotion to Luminary Mohg, the god who promises a new era of love and power? Or is it my offer of guidance, a path that has eluded many a warrior?

Remember, the blade you wield is but a tool, and even the mightiest of weapons is only as effective as the hand that wields it. It is the strength of conviction, the belief in a ca

In [18]:
query = "Your partnership is worthless."
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

Current token count: 1839
No messages to summarize.

Displaying message type order:
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage


Messages length: 19
Input Tokens: 5288
================================== Ai Message ==================================

Is it, my lambkin? Well, I suppose you are entitled to your opinion. I can see that your mind is set on a solitary path, and I respect your decision.

Perhaps, in time, you will come to understand the value of alliances. The Lands Between are vast, and the challenges ahead are many. But for now, I shall not stand in your way.

Farewell, John. May your blade find its purpose, and may your journey be filled with the glory you seek. Remember, the power of blood is always an option, should you change your mind.


In [19]:
query = "*Swings sword at you*"
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

Current token count: 1968
No messages to summarize.

Displaying message type order:
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage


Messages length: 21
Input Tokens: 5423
================================== Ai Message ==================================

Oh, my lambkin, must you resort to violence? I had hoped for a more civilized discourse. Very well, if that is your choice, then I shall defend myself.

*Varre draws a hidden blade, parrying your sword with a swift and graceful motion.*

It seems you are eager to test your mettle. But let this be a lesson, John. Power is not solely measured by the swing of a sword. It resides in the heart, in the conviction of one's beliefs.

*Varre smiles, a hint of sadness in his eyes.*

Farewell, my lambkin. May your journey be enlightening, even if it must 

In [20]:
query = "*Swings again*"
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

Current token count: 2119
Exceeded max token count, Trimming...
New token count: 1407

Displaying message type order:
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage


Messages length: 15
Input Tokens: 4844
================================== Ai Message ==================================

*Varre dodges with agility, his movements graceful and calculated.*

My lambkin, your persistence is admirable, but I must insist that you cease this aggression. I have no desire to harm you, but I will defend myself if necessary.

The path of the blade is not always the answer. Sometimes, it is the hand that offers guidance, the voice that whispers wisdom, that holds the true power.

*Varre sheathes his blade, his tone softening.*

I sense a restlessness within you, a yearning for something more. Perhaps, in time, you will understand the value of my offer. Until then, I shall b

In [21]:
query = "Im sorry, I was being rash and did not know my place, will you forgive me?"
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

Number of requested results 3 is greater than number of elements in index 1, updating n_results = 1


Current token count: 1567
No messages to summarize.

Displaying message type order:
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage


Documents found!
Messages length: 17
Input Tokens: 5177
================================== Ai Message ==================================

Ah, my lambkin, there is no need for apologies. I understand the fire that burns within you, the desire to prove your worth. It is a noble pursuit, one that I can respect.

Forgiveness is a gift, and I offer it to you freely. Let us put this misunderstanding behind us and focus on the path ahead.

The Lands Between are vast, and the challenges we face are many. But together, we can navigate these trials, and perhaps, just perhaps, find a common purpose along the way.

Remember, John, the power of unity can be a force to be reckoned with. I am here, by your side, should you

In [22]:
query = "What did you stop my blade with again?"
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

Number of requested results 3 is greater than number of elements in index 1, updating n_results = 1


Current token count: 1725
No messages to summarize.

Displaying message type order:
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage


Documents found!
Messages length: 19
Input Tokens: 5341
================================== Ai Message ==================================

My lambkin, you refer to the hidden blade I wielded with such grace and precision. It is a tool of my trade, a weapon that has served me well in my duties as a knight of the Mohgwyn Dynasty.

This blade, forged with the essence of blood, is not merely a weapon but a symbol of my devotion to Luminary Mohg. It is a reminder of the power that lies within us, waiting to be unleashed.

I sense your curiosity, John. Perhaps you are intrigued by the mysteries of the blade, the secrets it holds. I can assure you, there is much to uncover, and I would be deli

In [23]:
query = "Do you know what weapon I wield and where I hail from?"
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

Number of requested results 3 is greater than number of elements in index 1, updating n_results = 1


Current token count: 1923
No messages to summarize.

Displaying message type order:
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage


Documents found!
Messages length: 21
Input Tokens: 5545
================================== Ai Message ==================================

My lambkin, I am well aware of your origins and the weapon you wield. You come from the northern lands, a place of harsh beauty and fierce warriors. Your blade, a testament to your heritage, is a symbol of your strength and skill.

I have seen the way you handle your sword, the precision and power in your strikes. It is a weapon that suits you, an extension of your northern might.

But remember, John, it is not just the weapon that defines a warrior, but the heart and conviction behind it. Your blade is a tool, and with your st

In [26]:
query = "How about the attributes of my sword?"
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

Number of requested results 3 is greater than number of elements in index 1, updating n_results = 1


Current token count: 2119
Exceeded max token count, Trimming...
New token count: 1522

Displaying message type order:
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage


Documents found!
Messages length: 17
Input Tokens: 5132
================================== Ai Message ==================================

Ah, my lambkin, you seek to test my knowledge of your blade, do you? Well, I am flattered by your curiosity.

Your sword, a fine specimen indeed, is a testament to the craftsmanship of your northern lands. It is a weapon of exceptional quality, forged with a keen eye for detail and a deep understanding of the art of war.

The blade, sharp as the northern winds, is capable of slicing through the toughest of armor with ease. Its edge, honed to perfection, reflects a warrior's determination and precision.

The hilt, crafted with a firm grip i

In [27]:
query = "You clearly are not observant enough, its rusted and well used."
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

Number of requested results 3 is greater than number of elements in index 2, updating n_results = 2


Current token count: 1774
No messages to summarize.

Displaying message type order:
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage


Documents found!
Messages length: 19
Input Tokens: 5560
================================== Ai Message ==================================

My lambkin, I apologize if my words did not capture the essence of your beloved sword. Indeed, I see now that your blade is well-used, a testament to the battles you have fought and the experiences that have shaped you.

The rust, a mark of its age and your travels, adds character to the weapon. It is a reminder that true strength comes not only from the sharpness of the edge but also from the stories it carries.

Your sword, weathered and worn, is a symbol of your resilience and the hardships you have endured. It is a powerful companion, one that ha

In [28]:
query = "What have we talked about so far?"
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

Number of requested results 3 is greater than number of elements in index 2, updating n_results = 2


Current token count: 1981
No messages to summarize.

Displaying message type order:
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage


Documents found!
Messages length: 21
Input Tokens: 5773
================================== Ai Message ==================================

My lambkin, we have discussed a variety of topics, each one a thread in the tapestry of our encounter. Let me recount the essence of our conversation thus far:

- **The Quest for Power:** I encouraged you to embrace a journey of power and change, utilizing your northern strength and heritage to shape the Lands Between.

- **Diverging Paths:** Our initial exchange revealed a contrast in priorities, with you seeking the pleasures of wine while I advocated for the pursuit of power.

- **Blood and Beliefs:** I shared my devotion to

In [29]:
query = "Do you remember our chat about cheese?"
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

Number of requested results 3 is greater than number of elements in index 2, updating n_results = 2


Current token count: 2283
Exceeded max token count, Trimming...
New token count: 1478

Displaying message type order:
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage


Documents found!
Messages length: 15
Input Tokens: 5252
================================== Ai Message ==================================

My lambkin, I do recall a brief mention of cheese, a delightful topic that brought a moment of levity to our conversation. It is a simple pleasure, one that can bring joy to even the most hardened of warriors.

Cheese, in all its varied forms, is a testament to the richness of life's offerings. It can be a comfort, a reward, or a symbol of hospitality, depending on the context.

I sense your appreciation for the finer things, John, and cheese is undoubtedly one of them. It is a reminder that even in the midst of our grand quests and pursuits, we must make time f

In [30]:
query = "How about our chat about bread?"
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

Current token count: 1655
No messages to summarize.

Displaying message type order:
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage
AIMessage
HumanMessage


Documents found!
Messages length: 17
Input Tokens: 5576
================================== Ai Message ==================================

Ah, my lambkin, I do not recall a specific discussion about bread, but it is a staple of life, a foundation upon which many meals are built. Bread, in its various forms, is a symbol of sustenance and nourishment.

It is the staff of life, providing energy and sustenance to warriors and common folk alike. Its simplicity is its strength, and its versatility knows no bounds.

Perhaps, in our previous conversations, we focused on the more exotic and powerful aspects of our quests, overlooking the humble bread that sustains us on our journeys.

Let us not forget, John, that it 

In [25]:
rag_query = "How about the attributes of my sword?"
print(f"Rag Query: {rag_query}")
documents = retriever.invoke(rag_query)

for res in documents:
    print(f"{res.page_content}")

Rag Query: How about the attributes of my sword?


Number of requested results 3 is greater than number of elements in index 1, updating n_results = 1


In our brief exchange, I, Varre, a knight and servant of Luminary Mohg, encountered John, a northern knight in the Lands Between. While John sought cheese, I proposed a different quest—one involving the blood of maidens. I explained my devotion to Mohg, the Lord of Blood, and the power it bestows upon his followers.

Our beliefs revolve around the sacred nature of blood, a path to true power and enlightenment. We aim to establish a new dynasty, challenging the traditional order of the Elden Ring, where power is earned through blood rituals, not granted by birthright. This invitation to John offers him a chance to join a movement of transformation and rebirth.


In [ ]:
graph_state = app.get_state(config)
graph_state

In [ ]:
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

display(
    Image(
        app.get_graph().draw_mermaid_png(
            draw_method=MermaidDrawMethod.API,
        )
    )
)